In [1]:
CONTEXT_SIZE = 2 # 依据的单词数
EMBEDDING_DIM = 10 # 词向量的维度
# 我们使用莎士比亚的诗
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [2]:
trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2])
            for i in range(len(test_sentence)-2)]

In [3]:
len(trigram)

113

In [4]:
trigram[0]

(('When', 'forty'), 'winters')

In [5]:
vocb = set(test_sentence)
word2idx = {word: i for i, word in enumerate(vocb)}
idx2word = {word2idx[word]: word for word in word2idx}

In [6]:
word2idx

{'Will': 0,
 'This': 1,
 'succession': 2,
 'a': 3,
 'livery': 4,
 'say,': 5,
 'being': 6,
 'cold.': 7,
 "feel'st": 8,
 'much': 9,
 'When': 10,
 'sum': 11,
 'If': 12,
 'old': 13,
 'count,': 14,
 'where': 15,
 'thou': 16,
 'to': 17,
 'blood': 18,
 'weed': 19,
 'so': 20,
 'lusty': 21,
 'treasure': 22,
 'brow,': 23,
 "deserv'd": 24,
 'worth': 25,
 'warm': 26,
 'proud': 27,
 'the': 28,
 'made': 29,
 'shall': 30,
 'all': 31,
 'mine': 32,
 'Shall': 33,
 'were': 34,
 'see': 35,
 'lies,': 36,
 'couldst': 37,
 'small': 38,
 'Were': 39,
 'on': 40,
 'in': 41,
 'an': 42,
 'more': 43,
 'use,': 44,
 'praise': 45,
 'Proving': 46,
 'field,': 47,
 'it': 48,
 'Thy': 49,
 'beauty': 50,
 'And': 51,
 'be': 52,
 'old,': 53,
 'and': 54,
 'art': 55,
 'Then': 56,
 'by': 57,
 'thine!': 58,
 'days;': 59,
 'gazed': 60,
 'my': 61,
 'winters': 62,
 "totter'd": 63,
 'new': 64,
 'To': 65,
 "beauty's": 66,
 'dig': 67,
 'held:': 68,
 'now,': 69,
 'forty': 70,
 'sunken': 71,
 'asked,': 72,
 'within': 73,
 'thy': 74,
 "ex

In [7]:
import torch
from torch import nn
from torch.autograd import Variable

In [8]:
class n_gram(nn.Module):
    def __init__(self, vocb_size, context_size=CONTEXT_SIZE, n_dim=EMBEDDING_DIM):
        super(n_gram, self).__init__()
        self.embed = nn.Embedding(vocb_size, n_dim)
        self.classify = nn.Sequential(
            nn.Linear(context_size * n_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, vocb_size)
        )
    
    def forward(self, x):
        voc_embed = self.embed(x)
        voc_embed = voc_embed.view(1, -1)
        out = self.classify(voc_embed)
        return out

In [9]:
net = n_gram(len(word2idx))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-2, weight_decay=1e-5)

In [12]:
for e in range(100):
    train_loss = 0
    for word, label in trigram:
        word = Variable(torch.LongTensor([word2idx[i] for i in word]))
        label = Variable(torch.LongTensor([word2idx[label]]))
        # forward
        out = net(word)
        loss = criterion(out, label)
        train_loss += loss.item()
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # display
    if (e + 1) % 20 == 0:
        print('Epoch: {}, Loss: {:.6f}'.format(e + 1, train_loss / len(trigram)))

Epoch: 20, Loss: 0.058586
Epoch: 40, Loss: 0.054333
Epoch: 60, Loss: 0.051058
Epoch: 80, Loss: 0.048464
Epoch: 100, Loss: 0.046367


In [13]:
net = net.eval()

In [14]:
word, label = trigram[19]
print('input: {}'.format(word))
print('label: {}\n'.format(label))
word = Variable(torch.LongTensor([word2idx[i] for i in word]))
out = net(word)
pred_label_idx = out.max(1)[1].item()
predicted_word = idx2word[pred_label_idx]
print('read word is {}, predicted word is {}'.format(label, predicted_word))

input: ('so', 'gazed')
label: on

read word is on, predicted word is on


In [15]:
word, label = trigram[75]
print('input: {}'.format(word))
print('label: {}\n'.format(label))
word = Variable(torch.LongTensor([word2idx[i] for i in word]))
out = net(word)
pred_label_idx = out.max(1)[1].item()
predicted_word = idx2word[pred_label_idx]
print('read word is {}, predicted word is {}'.format(label, predicted_word))

input: ("'This", 'fair')
label: child

read word is child, predicted word is child
